<a href="https://colab.research.google.com/github/Feru34/Deep_Learning/blob/main/Talleres/Taller%202/Taller_2_%7BRueda_Felipe%7D_%7BMijares_Alfredo%7D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/fcastellanosp/MINE-4210_202420_ADL/blob/main/Talleres/Taller%202/MINE4210_ADL2024_Taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo ADL](https://github.com/fcastellanosp/MINE-4210_202420_ADL/blob/main/Laboratorios/logo_adl.png?raw=true)

# **Taller 2**

- **Integrante 1:** Felipe Rueda
- **Integrante 2:**

Bangladesh es una de los países con más tráfico en el mundo. Allí se encuentran algunas particularidades como los *rickshaw*, que son vehículos de 2 ruedas impulsados por una persona.

Los estudios indican que alrededos de 3.000 personas fallecen al año a causa de accidentes de tránsito.

Como consultor del sector de transporte, la organización ha procesado alrededor de 20.000 imágenes etiquetadas con información de objetos que pertenecen a las siguientes 13 clases:

* person
* rickshaw
* rickshaw van
* auto rickshaw
* truck
* pickup truck
* private car
* motorcycle
* bicycle
* bus
* micro bus
* covered van
* human hauler

Han sido contratados para buscar un proceso que permita acelerar el entendimiento de esta problemática, para el cuál le solicitan construir un modelo de detección de objetos que permita identificar los diferentes actores viales.

La información etiquetada se encuentra en  "[Bangladesh traffic](https://www.kaggle.com/datasets/hasibzunair/rsud20k-bangladesh-road-scene-understanding)".

# **Preguntas:**

1.   Construir una red neuronal de detección de objetos usando un modelo pre-entrenado. Recuerden ser descriptivos en el Preprocesamiento de Datos, Utilización de Modelos Pre-entrenados, Evaluación de Rendimiento y búsqueda de hiperparámetros.

2.  Proporcionar algunos ejemplos de detección de objetos con la red entrenada y lo comparen con la imagen con los datos anotados. Traten de usar imágenes con 1 objeto asi como imágenes con varios objetos.

3.  Investigar la métrica mAP y cómo la interpreta para la red entrenada.

4. Calcular el número de escenarios (imágenes) que involucran personas en el conjunto de prueba.

# **Notas Importantes:**
- Pueden usar la versión de Yolo del laboratorio u otra distinta.
- El archivo a presentar debe ser en formato .ipynb o HTML ya ejecutado. Celda que no esté ejecutada no se podrá evaluar.
- El nombre del archivo debe ser `Taller_2_{Apellido_Nombre}_{Apellido_Nombre}` de cada integrante respectivamente.

# 1. Construir una red neuronal de detección de objetos usando un modelo pre-entrenado

## Utilización de librerías

In [ ]:

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
import pandas as pd
import numpy as np

import os
import os.path as osp
import random
import re
import shutil

#Visualización de datos
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from IPython.display import display, Image

from numpy import asarray

from PIL import Image

from ultralytics import YOLO

In [ ]:
!nvidia-smi

## Preprocesamiento de datos

In [ ]:
my_seed = 19 #Semilla para reproducibilidad
IMG_EXT = ['.jpg', '.jpeg', '.png', '.bmp'] #Extensiones válidas de imagen
TEXT_EXT = '.txt'
YOLO_VERSION = 'yolov8'

## Utilización de Modelos Pre-entrenados

## Evaluación de Rendimiento

## Búsqueda de hiperparámetros

# Ejemplos de detección de objetos con la red entrenada y su respetiva comparación

# Métrica mAP

## Cómo la interpreta para la red entrenada?

# Número de escenarios (imágenes) que involucran personas en el conjunto de prueba